In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
import geopandas as gpd
import pickle


In [ ]:
# Only set to True for co-dev of ela from this use case:
ela_from_source = False
ela_from_source = True

In [ ]:
if ela_from_source:
    if ('ELA_SRC' in os.environ):
        root_src_dir = os.environ['ELA_SRC']
    elif sys.platform == 'win32':
        root_src_dir = r'C:\src\github_jm\pyela'
    else:
        username = os.environ['USER']
        root_src_dir = os.path.join('/home', username, 'src/ela/pyela')
    pkg_src_dir = root_src_dir
    sys.path.insert(0, pkg_src_dir)

In [ ]:
from ela.textproc import *
from ela.utils import *
from ela.classification import *
from ela.visual import *
from ela.visual3d import *
from ela.spatial import SliceOperation

## Importing data


In [ ]:
data_path = None

You probably want to explicitly set `data_path` to the location where you put the folder(s) e.g:

In [ ]:
#data_path = '/home/myusername/data' # On Linux, if you now have the folder /home/myusername/data/Bungendore
#data_path = r'C:\data\Lithology'  # windows, if you have C:\data\Lithology\Bungendore

Otherwise a fallback for the pyela developer(s)

In [ ]:
if data_path is None:
    if ('ELA_DATA' in os.environ):
        data_path = os.environ['ELA_DATA']
    elif sys.platform == 'win32':
        data_path = r'C:\data\Lithology'
    else:
        username = os.environ['USER']
        data_path = os.path.join('/home', username, 'data')

In [ ]:
data_path

In [ ]:
aem_datadir = os.path.join(data_path, 'AEM')
swan_datadir = os.path.join(data_path, 'swan_coastal')
scp_datadir = os.path.join(aem_datadir, 'Swan_coastal_plains')
scp_grids_datadir = os.path.join(scp_datadir, 'grids')
ngis_datadir = os.path.join(data_path, 'NGIS')
scp_shp_datadir = os.path.join(data_path, 'NGIS/swan_coastal')

## reload processed data


In [ ]:
dem = rasterio.open(os.path.join(swan_datadir,'Swan_DEM/CLIP.tif'))

In [ ]:
cnd_slice_dir = os.path.join(scp_grids_datadir,'cnd')
cnd_000_005 = rasterio.open(os.path.join(cnd_slice_dir,'Swan_Coastal_Plain_CND_000m_to_005m_Final.ers'))

In [ ]:
_, ax = plt.subplots(figsize=(12, 12))
show(cnd_000_005,title='Conductivity 0-5 metres depth (units?)', cmap='viridis',  ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
show(cnd_000_005,title='Conductivity 0-5 metres depth (units?)', cmap='magma',  ax=ax)

In [ ]:
bore_locations_raw = gpd.read_file(os.path.join(scp_shp_datadir, 'scp.shp'))

The DEM raster and the bore location shapefile do not use the same projection (coordinate reference system) so we reproject one of them. We choose the raster's UTM.

In [ ]:
bore_locations = bore_locations_raw.to_crs(dem.crs)

### Subset to the location of interest

The lithology logs are for all of western australia, which is much larger than the area of interest and for which we have the geolocation of boreholes. We subset to the location of interest 

In [ ]:
DEPTH_FROM_COL = 'FromDepth'
DEPTH_TO_COL = 'ToDepth'

TOP_ELEV_COL = 'TopElev'
BOTTOM_ELEV_COL = 'BottomElev'

LITHO_DESC_COL = 'Description'
HYDRO_CODE_COL = 'HydroCode'

In [ ]:
# to be reused in experimental notebooks:
interp_litho_filename = os.path.join(swan_datadir,'3d_primary_litho.pkl')
with open(interp_litho_filename, 'rb') as handle:
    lithology_3d_array = pickle.load(handle)

In [ ]:
lithologies = ['sand', 'clay','quartz','shale','sandstone', 'coal','pebbles','silt','pyrite','grit','limestone']

And to capture any of these we devise a regular expression:

In [ ]:
any_litho_markers_re = r'sand|clay|quart|ston|shale|silt|pebb|coal|pyr|grit|lime'
regex = re.compile(any_litho_markers_re)

In [ ]:
my_lithologies_numclasses = create_numeric_classes(lithologies)

In [ ]:
lithologies_dict = dict([(x,x) for x in lithologies])
lithologies_dict['sands'] = 'sand'
lithologies_dict['clays'] = 'clay'
lithologies_dict['shales'] = 'shale'
lithologies_dict['claystone'] = 'clay'
lithologies_dict['siltstone'] = 'silt'
lithologies_dict['limesand'] = 'sand' # ??
lithologies_dict['calcarenite'] = 'limestone' # ??
lithologies_dict['calcitareous'] = 'limestone' # ??
lithologies_dict['mudstone'] = 'silt' # ??
lithologies_dict['capstone'] = 'limestone' # ??
lithologies_dict['ironstone'] = 'sandstone' # ??
#lithologies_dict['topsoil'] = 'soil' # ??

In [ ]:
lithologies_adjective_dict = {
    'sandy' :  'sand',
    'clayey' :  'clay',
    'clayish' :  'clay',
    'shaley' :  'shale',
    'silty' :  'silt',
    'pebbly' :  'pebble',
    'gravelly' :  'gravel'
}

In [ ]:

fp = os.path.join(swan_datadir, 'dem_array_data.pkl')
with open(fp, 'rb') as handle:
    dem_array_data = pickle.load(handle)

## 2D visualisations

In [ ]:
lithology_color_names = ['yellow', 'olive', 'lightgrey', 'dimgray', 'teal',  'cornsilk',     'saddlebrown', 'rosybrown', 'chocolate', 'lightslategrey', 'gold']

In [ ]:
lithology_cmap = discrete_classes_colormap(lithology_color_names) # Later for exporting to RGB geotiffs??
litho_legend_display_info = [(lithology_cmap[i], lithologies[i], lithology_color_names[i]) for i in range(len(lithologies))]

In [ ]:
litho_legend = legend_fig(litho_legend_display_info)

In [ ]:
cms = cartopy_color_settings(lithology_color_names)

In [ ]:
dem_array_data.keys()

In [ ]:
ahd_min=-180
ahd_max=50

z_ahd_coords = np.arange(ahd_min,ahd_max,1)
dim_x,dim_y,dim_z = lithology_3d_array.shape
dims = (dim_x,dim_y,dim_z)

In [ ]:
# Burn DEM into grid
z_index_for_ahd = z_index_for_ahd_functor(b=-ahd_min)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
imgplot = plt.imshow(to_carto(lithology_3d_array[:, :, z_index_for_ahd(0)]), cmap=cms['cmap'])
title = plt.title('Primary litho at +0mAHD')

## 3D visualisation

In [ ]:
from ela.visual3d import *

In [ ]:
from mayavi import mlab

In [ ]:
vis_litho = LithologiesClassesVisual3d(lithologies, lithology_color_names, 'black')

In [ ]:
vis_litho.render_classes_planar(lithology_3d_array, 'Primary lithology')

ela has facilities to visualise overlaid information: DEM, classified bore logs, and volumes of interpolated lithologies. This is important to convey .

First a bit of data filling for visual purposes, as NaN lithology class codes may cause issues.

In [ ]:

classified_logs_filename = os.path.join(swan_datadir, 'classified_logs.pkl')
with open(classified_logs_filename, 'rb') as handle:
    df = pickle.load(handle)

In [ ]:
df_infilled = df.fillna({PRIMARY_LITHO_NUM_COL: -1.0})
# df_2 = df_1[(df_1[DEPTH_TO_AHD_COL] > (ahd_min-20))]

In [ ]:
# A factor to apply to Z coordinates, otherwise things would be squashed visually along the heights.
# Would prefer a visual only scaling factor, but could not find a way to do so. 
Z_SCALING = 20.0

In [ ]:
z_coords = np.arange(ahd_min,ahd_max,1)

In [ ]:
overlay_vis_litho = LithologiesClassesOverlayVisual3d(lithologies, lithology_color_names, 'black', dem_array_data, z_coords, Z_SCALING, df_infilled, PRIMARY_LITHO_NUM_COL)

In [ ]:
def view_class(value):
    f = overlay_vis_litho.view_overlay(value, lithology_3d_array)
    return f

In [ ]:
f = view_class(1.0)

In [ ]:
f = view_class(2.0)

![3D Interpolated overlay primary lithology quartz](img/snapshot_quartz.png)

In [ ]:
vis_litho = LithologiesClassesVisual3d(lithologies, lithology_color_names, 'black')

In [ ]:
vis_litho.render_classes_planar(lithology_3d_array, 'Primary lithology')